In [1]:
#  시스템콜 이름 -> 시스템콜 번호 딕셔너리 만들기
import subprocess
sysname_convert = dict()
sysnum_convert = dict()
with open("/tmp/syscall_64.txt","r") as f:
    for line in f.readlines():
        sysnum = line.strip().split()[0] # sysnum
        if len(line.strip().split()) == 4:
            sysname = line.strip().split()[3] # sysname
            if "compat_sys_" in sysname:
                sysname = sysname.replace("compat_sys_","")
            elif "sys_" in sysname:
                sysname = sysname.replace("sys_","")
            if "stub_x32_" in sysname:
                sysname = sysname.replace("stub_x32_","")
            if "stub_" in sysname:
                sysname = sysname.replace("stub_","")
        else:
            sysname = line.strip().split()[2] # sysname
        # x86 기준, x86 시스템콜에 없는 번호는 x32 기준
        if not sysname in sysname_convert.keys():
            sysname_convert[sysname] = sysnum
            sysnum_convert[sysnum] = sysname

In [2]:
# 기존에 없는 시스템콜 처리
sysname_convert["socketcall"] = "437"
sysnum_convert["437"] = "socketcall"
sysname_convert["vm86old"] = "436"
sysnum_convert["436"] = "vm86old"

# Benign Image

In [3]:
import subprocess

# 모든 benign syscall sequence load
seqList = dict()
imgList = {"httpd", "mongodb", "mysql", "nginx", "redis", "mariadb", "node", "tomcat", "gcc", "openjdk", "gzip", "bzip2", "ghostscript", "lowriter", "qalc"}

for imgName in imgList:
    seqs = subprocess.check_output(f"find ../data/parsing/{imgName}/*.txt", shell=True).decode().strip().split()
    for seq in seqs:
        case = seq.replace(f"../data/parsing/{imgName}/","").replace(".txt","")
        with open(seq, "r") as f:
            syslist = f.readlines()
            syslist = list(map(lambda x: x.strip(), syslist))
            seqList[f"{imgName}_{case}"]=syslist
        
print(len(seqList))

811


In [4]:
# 시스템콜 번호으로 바꾸기
seqNumList = dict()

for case, seq in seqList.items():
    # 변환
    num_seq = list()
    for sys_name in seq:
        num_seq.append(sysname_convert[sys_name])
    seqNumList[case]=num_seq

In [7]:
import pickle

# pickle로 저장
with open('seqNumList.pkl', 'wb') as handle:
    pickle.dump(seqNumList, handle, protocol=pickle.HIGHEST_PROTOCOL)

## System Call Set 분석

In [10]:
# System Call Set 분석
b_sysSet = dict()
for imgName in imgList:
    b_sysSet[imgName] = set()

for case, seq in seqNumList.items():
    imgName = case.split("_")[0]
    for syscall in seq:
        b_sysSet[imgName].add(syscall)
        
b_sysSet_total = set()
for imgName in imgList:
    print(f"image {imgName} syscall set #: {len(b_sysSet[imgName])}")
    b_sysSet_total = b_sysSet_total | b_sysSet[imgName]
print(f"total syscall set #: {len(b_sysSet_total)}")

image httpd syscall set #: 21
image node syscall set #: 13
image tomcat syscall set #: 25
image lowriter syscall set #: 76
image mariadb syscall set #: 22
image bzip2 syscall set #: 17
image qalc syscall set #: 42
image mysql syscall set #: 39
image mongodb syscall set #: 45
image redis syscall set #: 14
image ghostscript syscall set #: 38
image nginx syscall set #: 14
image gcc syscall set #: 38
image gzip syscall set #: 20
image openjdk syscall set #: 57
total syscall set #: 107


In [11]:
import pickle

# pickle로 저장
with open('b_sysSet.pkl', 'wb') as handle:
    pickle.dump(b_sysSet, handle, protocol=pickle.HIGHEST_PROTOCOL)

## 반복 패턴 처리

In [7]:
# N-gram 생성 함수
def tokenize(trg, N=1):
    a = trg.split(",")                 
    d = []
    for i in range(len(a) - N+1):
        b = a[i:i+N]
        c = (',').join(b)
        d.append(c)
    return d

In [8]:
# 3개 이상 반복 패턴 찾기
def find_pattern(ex):    
    all_pattern = set()
    # make ngram
    ngrams = set()
    for i in range(2, len(ex.split(","))):
        for token in tokenize(ex, i):
            ngrams.add(token)
    # find ngram pattern
    for ngram in ngrams:
        replaced_ex = ex.replace(ngram,"*")
        if "*,*,*" in replaced_ex:
            all_pattern.add(ngram)
    return  all_pattern

In [9]:
with open("all_patterns.txt") as f:
    sysList = f.readlines()
    
all_pattern = set()
for seq in sysList:
    all_pattern = all_pattern | find_pattern(seq.strip())

print(all_pattern)

{'12,12', '5,0,257,4,12,12', '257,4,5,0', '321,1,321,321,321', '233,232,232', '3,44,21,21,41,16', '257,4,12,12,5,0', '12,12,5,0,257,4', '33,233,3', '56,61', '4,257', '0,3,5,9,10,12,13,14,17,56,218,257,273,302', '250,230', '21,41,16,3,44,21', '0,3,5,9,10,12,13,14,17,56,218,257,273,302,1,230', '257,4', '16,3,44,21,21,41', '53,44', '5,0,5,0', '257,4,257,4', '5,0,257,4', '5,0', '21,21,41,16,3,44', '321,321,1,321,321', '1,321,321,321,321', '22,3,3', '4,230,257', '321,321,321,1,321', '12,5,0,257,4,12', '257,4,230', '54,54', '41,16,3,44,21,21', '101,101', '44,21,21,41,16,3', '0,257,4,12,12,5', '0,5', '3,33,233', '230,0,3,5,9,10,12,13,14,17,56,218,257,273,302,1', '230,257,4', '233,232', '0,257,4,5', '321,321,321,321,1', '230,250', '257,1,3'}


In [10]:
all_pattern_list = list(all_pattern)
all_pattern_list.sort(key=len)

iter_seqNumList = dict()
for testcase in seqNumList.keys():
    print(testcase)
    seq = ",".join(seqNumList[testcase])
    for pattern in all_pattern_list:
        if pattern in seq:
            seq = seq.replace(pattern, "*")
            if "*,*" in seq:
                for i in range(seq.count("*")):
                    seq = seq.replace("*,*","*")
                seq = seq.replace("*", pattern+","+pattern+","+pattern)
                break
            else:
                seq = seq.replace("*", pattern)
    iter_seqNumList[testcase] = seq.split(",")

gcc_strace1-1382943
gcc_strace1-1382990
gcc_strace1-1383103
gcc_strace1-1383104
gcc_strace1-1383105
gcc_strace1-1383106
gcc_strace1-1383107
gcc_strace2-1386374
gcc_strace2-1386420
gcc_strace2-1386531
gcc_strace2-1386532
gcc_strace2-1386533
gcc_strace2-1386534
gcc_strace2-1386535
mysql_c-3201775
mysql_c-3201908
mysql_c-3201909
mysql_c-3201910
mysql_c-3201911
mysql_c-3201912
mysql_c-3201913
mysql_c-3201914
mysql_c-3201915
mysql_c-3201916
mysql_c-3201917
mysql_c-3201918
mysql_c-3201920
mysql_c-3201921
mysql_c-3201923
mysql_c-3201929
mysql_c-3202244
mysql_c-3202245
mysql_c-3202246
mysql_c-3202247
mysql_c-3202248
mysql_c-3202249
mysql_c-3202250
mysql_c-3202251
mysql_c-3202252
mysql_c-3202253
mysql_c-3202254
mysql_c-3202256
mysql_c-3202257
mysql_c-3202259
mysql_c-3203576
mysql_d-3703372
mysql_d-3703520
mysql_d-3703521
mysql_d-3703522
mysql_d-3703523
mysql_d-3703524
mysql_d-3703525
mysql_d-3703526
mysql_d-3703527
mysql_d-3703528
mysql_d-3703529
mysql_d-3703530
mysql_d-3703532
mysql_d-3703533


lowriter_doctopdf-2058397
lowriter_doctopdf-2058398
lowriter_doctopdf-2058399
lowriter_doctopdf-2058400
lowriter_doctopdf-2058401
lowriter_doctopdf-2058402
lowriter_doctopdf-2058405
lowriter_encodeUTF8-2059660
lowriter_encodeUTF8-2059724
lowriter_encodeUTF8-2059734
lowriter_encodeUTF8-2059736
lowriter_encodeUTF8-2059738
lowriter_encodeUTF8-2059740
lowriter_encodeUTF8-2059741
lowriter_encodeUTF8-2059742
lowriter_encodeUTF8-2059743
lowriter_encodeUTF8-2059744
lowriter_encodeUTF8-2059745
lowriter_encodeUTF8-2059746
lowriter_encodeUTF8-2059747
lowriter_encodeUTF8-2059748
lowriter_encodeUTF8-2059749
lowriter_encodeUTF8-2059750
lowriter_encodeUTF8-2059751
lowriter_encodeUTF8-2059752
lowriter_encodeUTF8-2059756
lowriter_encodeUTF8-2059757
lowriter_encodeUTF8-2059758
lowriter_encodeUTF8-2059759
lowriter_encodeUTF8-2059760
lowriter_encodeUTF8-2059765
lowriter_encodeUTF8-2059766
lowriter_encodeUTF8-2059767
lowriter_encodeUTF8-2059769
lowriter_encodeUTF8-2059770
lowriter_encodeUTF8-2059771
lowrit

# Exploit(106)과 Benign의 Set 분석

## Exploit Code N-gram

In [11]:
import pickle

## Load pickle
# with open("/opt/output/perm/analysis/soft_dedup_ngramdf.pkl","rb") as f:
with open("soft_dedup_ngramdf.pkl","rb") as f:
    m_ngram = pickle.load(f)
m_ngram

,N,Ngram_syscall,num_of_exploit,EIDs
0,61,"[41, 248, 248, 248, 54, 0, 3, 5, 9, 10, 12, 13...",2,"[40871, 47170]"
1,49,"[230, 0, 3, 5, 9, 10, 12, 13, 14, 17, 56, 218,...",2,"[40871, 47170]"
2,45,"[12, 12, 5, 0, 257, 4, 12, 12, 5, 0, 257, 4, 1...",2,"[47163, 50541]"
3,43,"[9, 0, 3, 5, 9, 10, 12, 13, 14, 17, 56, 218, 2...",2,"[40616, 43127]"
4,43,"[12, 12, 5, 0, 257, 4, 12, 12, 5, 0, 257, 4, 1...",2,"[47163, 50541]"
...,...,...,...,...
473,2,"[257, 0]",16,"[15024, 15774, 3, 35370, 40616, 40871, 42275, ..."
474,2,"[8, 0]",8,"[12, 20979, 27297, 3, 43127, 45553, 50808, 9083]"
475,2,"[3, 59]",9,"[39230, 40871, 43418, 47170, 6851, 9479, 9542]"
476,2,"[8, 5]",4,"[12, 3, 43127, 50541]"


## Exploit(106) Syscall Set

In [12]:
# 모든 익스플로잇 path laod
import subprocess
import json

pathFile = subprocess.check_output(f"find ../data/path/*.json", shell=True).decode().strip().split()
EIDList = list(map(lambda x: x.replace("../data/path/","").replace(".json",""), pathFile))
pathList = dict() # 익스플로잇 코드 path dictionary의 dictionary - pathList[EID][path] = path 파일 정보
for EID in EIDList:
    with open(f"../data/path/{EID}.json", "r") as f:
        pathList[EID] = dict()
        pathList[EID]["path"] = json.load(f)
        
# dict {EID: all path list} 만들어 주기
eid_all_path_dict = dict()
for EID, pathset in pathList.items():
    eid_all_path_dict[EID] = list()
    for userfunc, pathset in pathset['path'].items():
        if not len(pathset) == 0:
            eid_all_path_dict[EID].extend(pathset)
            
# 중복 제거
eid_all_path_set = dict()
for EID, pathset in eid_all_path_dict.items():
    eid_all_path_set[EID] = set()
    for path in pathset:
        eid_all_path_set[EID].add(",".join(path))
        
m_sysSet = set()
for EID, pathset in eid_all_path_set.items():
    for path in pathset:
        if path:
            m_sysSet = m_sysSet | set(path.split(","))
len(m_sysSet)

140

In [13]:
def false_positive_set(m, b):
    falsePositive_sysSet = m & b
    print(f"false positive syscall #: {len(falsePositive_sysSet)}")
    tmp = set()
    for syscall in falsePositive_sysSet:
        tmp.add(sysnum_convert[syscall])
    print(tmp)

In [14]:
# all
print("="*10,"<all image>","="*10)
false_positive_set(m_sysSet, b_sysSet_total)
for imgName in imgList:
    print("="*10,f"<{imgName}>","="*10)
    false_positive_set(m_sysSet, b_sysSet[imgName])

========== <all image> ==========
false positive syscall #: 80
{'socket', 'listen', 'munmap', 'epoll_ctl', 'fcntl', 'sendfile64', 'rmdir', 'getsockopt', 'timerfd_settime', 'pipe', 'chown', 'mmap', 'sendto', 'rt_sigreturn', 'recvfrom', 'newfstat', 'link', 'recvmsg', 'set_robust_list', 'geteuid', 'nanosleep', 'arch_prctl', 'shutdown', 'write', 'newfstatat', 'futex', 'sendmmsg', 'close', 'sendmsg', 'exit_group', 'clock_nanosleep', 'socketpair', 'newstat', 'ftruncate', 'openat', 'bind', 'mprotect', 'accept', 'rename', 'brk', 'epoll_wait', 'setrlimit', 'ioctl', 'newuname', 'dup', 'getppid', 'madvise', 'getpid', 'unlink', 'set_tid_address', 'getcwd', 'rt_sigprocmask', 'mkdir', 'chmod', 'getuid', 'execve', 'getgid', 'exit', 'read', 'lseek', 'prlimit64', 'rt_sigaction', 'getsockname', 'statfs', 'poll', 'sigaltstack', 'getdents64', 'setsockopt', 'wait4', 'getegid', 'readlink', 'prctl', 'gettid', 'pread64', 'umask', 'connect', 'open', 'access', 'clone', 'sched_yield'}
========== <gcc> ==========

# Token 크기 별 Overlapped N-gram 개수

In [57]:
import pandas as pd

overlap_ngram = pd.DataFrame(columns=["N","nginx","httpd","tomcat","node","mongodb","mysql","mariadb","redis",
                                      "gcc","openjdk","gzip","bzip2","qalc","ghostscript","lowriter","total"])

for N in set(m_ngram["N"].to_list()):
    overlaptmp = {"N":N, "nginx":set(),"httpd":set(),"tomcat":set(),"node":set(),"mongodb":set(),"mysql":set(),
                  "mariadb":set(),"redis":set(),"gcc":set(),"openjdk":set(),"gzip":set(),"bzip2":set(),"qalc":set(),
                  "ghostscript":set(),"lowriter":set(),"total":set()}
    overlap_ngram = overlap_ngram.append(overlaptmp, ignore_index=True)
overlap_ngram = overlap_ngram.set_index("N")

for index, row in m_ngram.iterrows():
    for testName, seq in iter_seqNumList.items():
        imgName = testName.split("_")[0]
        if ",".join(row.Ngram_syscall) in ",".join(seq):
            cur = overlap_ngram[imgName].loc[row.N]
            new = cur | {",".join(row.Ngram_syscall)}
            # new
            overlap_ngram.at[row.N,imgName] = new
            
overlap_ngram

,nginx,httpd,tomcat,node,mongodb,mysql,mariadb,redis,gcc,openjdk,gzip,bzip2,qalc,ghostscript,lowriter,total
N,,,,,,,,,,,,,,,,
2,"{257,5, 3,3, 1,3}","{257,257, 3,0, 257,5, 9,257, 257,9, 5,5, 0,8, ...","{8,1, 257,5, 8,5, 3,257, 54,54, 5,0, 0,3}","{8,1, 12,12, 3,3, 8,0}","{8,1, 257,5, 9,257, 257,0, 9,9, 3,257, 157,39,...","{8,1, 257,5, 257,8, 9,11, 1,0, 257,0, 9,44, 9,...","{8,1, 5,3, 54,54, 4,0, 202,257, 257,0, 8,5, 0,...","{39,257, 54,54, 9,257, 257,0, 8,5, 9,9, 257,3,...","{56,56, 3,61, 9,11, 13,14, 102,104, 3,3, 8,0, ...","{41,41, 257,4, 257,5, 9,257, 56,56, 9,11, 3,25...","{8,1, 0,0, 5,5, 1,0, 1,3, 9,9, 12,12, 3,3, 0,3}","{8,1, 0,0, 3,231, 1,0, 1,3, 9,9, 11,3, 12,12, ...","{41,41, 39,13, 1,102, 257,4, 257,5, 13,257, 9,...","{257,4, 257,5, 9,11, 1,0, 3,257, 257,3, 3,3, 8...","{41,41, 1,102, 257,4, 257,5, 9,11, 3,257, 22,5...",{}
3,"{5,1,3}","{9,9,9, 8,0,3}","{257,5,0, 3,257,5, 5,0,3}","{3,3,22, 1,1,1}","{0,0,3, 0,0,0, 3,3,1, 257,5,0, 3,257,5, 5,0,3}","{3,257,5, 8,8,8, 8,0,8, 8,5,0, 257,5,9, 257,0,...","{3,3,22, 257,0,3}","{3,3,22, 257,0,3, 54,54,54}","{0,8,8, 3,3,1, 0,0,3, 3,12,12, 12,12,9, 1,1,1,...","{0,8,8, 230,3,3, 202,202,230, 28,28,28, 3,3,1,...","{12,12,9, 1,1,1, 12,12,0, 12,12,1, 0,3,3, 102,...","{12,12,9, 1,1,1, 12,12,1, 16,16,16, 9,9,9, 0,0...","{0,8,8, 1,1,1, 12,12,0, 12,12,1, 257,257,0, 0,...","{12,12,5, 0,8,8, 12,12,3, 257,12,12, 3,3,1, 25...","{12,12,5, 0,8,8, 83,83,83, 22,22,22, 12,12,3, ...",{}
4,{},{},{},{},"{0,0,0,3}","{9,9,9,9, 9,3,257,0, 0,0,0,0, 0,0,0,3, 257,5,0,0}",{},{},"{12,12,12,12, 9,9,9,9, 3,3,3,3, 0,0,0,0, 0,0,0,3}","{12,12,12,12, 9,9,9,9, 3,3,3,3, 0,0,0,0, 0,0,0,3}","{12,12,12,12, 5,12,12,1, 0,0,0,0, 9,9,9,9}","{12,12,12,12, 9,9,9,9, 0,0,0,0}","{12,12,12,12, 9,9,9,9, 0,0,0,0, 17,17,17,17, 0...","{12,12,12,12, 9,9,9,9, 3,3,3,3, 0,0,0,0, 17,17...","{12,12,12,12, 5,12,12,1, 9,9,9,9, 3,3,3,3, 0,0...",{}
5,{},{},{},{},{},"{5,9,9,9,9, 0,0,0,0,3}",{},{},"{5,9,9,9,9, 9,9,9,9,9, 0,0,0,0,3}","{5,9,9,9,9, 9,9,9,9,9, 0,0,0,0,3}",{},"{9,9,9,9,9}","{5,9,9,9,9, 9,9,9,9,9, 0,0,0,0,3}","{257,0,0,0,0, 0,0,0,0,3}","{9,9,9,9,9, 5,9,9,9,9, 257,0,0,0,0, 0,0,0,0,3}",{}
6,{},{},{},{},{},{},{},"{54,54,54,54,54,54}","{12,12,12,12,12,12}","{5,9,9,9,9,9, 12,12,12,12,12,12}","{12,12,12,12,12,12}","{12,12,12,12,12,12}","{5,9,9,9,9,9, 12,12,12,12,12,12}","{12,12,12,12,12,12}","{5,9,9,9,9,9, 12,12,12,12,12,12}",{}
7,{},{},{},{},{},{},{},"{54,54,54,54,54,54,54}",{},{},{},{},{},{},{},{}
8,{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}
9,{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}
10,{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}


In [58]:
# total 구하기
for index, row in overlap_ngram.iterrows():
    tmpset = set()
    for imgName in imgList:
        tmpset = tmpset | overlap_ngram[imgName][index]
    overlap_ngram["total"][index] = tmpset
    
# pickle로 저장
overlap_ngram.to_pickle('overlap_ngram.pkl')
overlap_ngram

,nginx,httpd,tomcat,node,mongodb,mysql,mariadb,redis,gcc,openjdk,gzip,bzip2,qalc,ghostscript,lowriter,total
N,,,,,,,,,,,,,,,,
2,"{257,5, 3,3, 1,3}","{257,257, 3,0, 257,5, 9,257, 257,9, 5,5, 0,8, ...","{8,1, 257,5, 8,5, 3,257, 54,54, 5,0, 0,3}","{8,1, 12,12, 3,3, 8,0}","{8,1, 257,5, 9,257, 257,0, 9,9, 3,257, 157,39,...","{8,1, 257,5, 257,8, 9,11, 1,0, 257,0, 9,44, 9,...","{8,1, 5,3, 54,54, 4,0, 202,257, 257,0, 8,5, 0,...","{39,257, 54,54, 9,257, 257,0, 8,5, 9,9, 257,3,...","{56,56, 3,61, 9,11, 13,14, 102,104, 3,3, 8,0, ...","{41,41, 257,4, 257,5, 9,257, 56,56, 9,11, 3,25...","{8,1, 0,0, 5,5, 1,0, 1,3, 9,9, 12,12, 3,3, 0,3}","{8,1, 0,0, 3,231, 1,0, 1,3, 9,9, 11,3, 12,12, ...","{41,41, 39,13, 1,102, 257,4, 257,5, 13,257, 9,...","{257,4, 257,5, 9,11, 1,0, 3,257, 257,3, 3,3, 8...","{41,41, 1,102, 257,4, 257,5, 9,11, 3,257, 22,5...","{41,41, 1,102, 257,5, 257,4, 9,11, 1,0, 3,257,..."
3,"{5,1,3}","{9,9,9, 8,0,3}","{257,5,0, 3,257,5, 5,0,3}","{3,3,22, 1,1,1}","{0,0,3, 0,0,0, 3,3,1, 257,5,0, 3,257,5, 5,0,3}","{3,257,5, 8,8,8, 8,0,8, 8,5,0, 257,5,9, 257,0,...","{3,3,22, 257,0,3}","{3,3,22, 257,0,3, 54,54,54}","{0,8,8, 3,3,1, 0,0,3, 3,12,12, 12,12,9, 1,1,1,...","{0,8,8, 230,3,3, 202,202,230, 28,28,28, 3,3,1,...","{12,12,9, 1,1,1, 12,12,0, 12,12,1, 0,3,3, 102,...","{12,12,9, 1,1,1, 12,12,1, 16,16,16, 9,9,9, 0,0...","{0,8,8, 1,1,1, 12,12,0, 12,12,1, 257,257,0, 0,...","{12,12,5, 0,8,8, 12,12,3, 257,12,12, 3,3,1, 25...","{12,12,5, 0,8,8, 83,83,83, 22,22,22, 12,12,3, ...","{0,8,8, 83,83,83, 22,22,22, 8,5,0, 16,16,16, 2..."
4,{},{},{},{},"{0,0,0,3}","{9,9,9,9, 9,3,257,0, 0,0,0,0, 0,0,0,3, 257,5,0,0}",{},{},"{12,12,12,12, 9,9,9,9, 3,3,3,3, 0,0,0,0, 0,0,0,3}","{12,12,12,12, 9,9,9,9, 3,3,3,3, 0,0,0,0, 0,0,0,3}","{12,12,12,12, 5,12,12,1, 0,0,0,0, 9,9,9,9}","{12,12,12,12, 9,9,9,9, 0,0,0,0}","{12,12,12,12, 9,9,9,9, 0,0,0,0, 17,17,17,17, 0...","{12,12,12,12, 9,9,9,9, 3,3,3,3, 0,0,0,0, 17,17...","{12,12,12,12, 5,12,12,1, 9,9,9,9, 3,3,3,3, 0,0...","{9,9,9,9, 9,3,257,0, 0,0,0,0, 0,0,0,3, 12,12,1..."
5,{},{},{},{},{},"{5,9,9,9,9, 0,0,0,0,3}",{},{},"{5,9,9,9,9, 9,9,9,9,9, 0,0,0,0,3}","{5,9,9,9,9, 9,9,9,9,9, 0,0,0,0,3}",{},"{9,9,9,9,9}","{5,9,9,9,9, 9,9,9,9,9, 0,0,0,0,3}","{257,0,0,0,0, 0,0,0,0,3}","{9,9,9,9,9, 5,9,9,9,9, 257,0,0,0,0, 0,0,0,0,3}","{5,9,9,9,9, 257,0,0,0,0, 9,9,9,9,9, 0,0,0,0,3}"
6,{},{},{},{},{},{},{},"{54,54,54,54,54,54}","{12,12,12,12,12,12}","{5,9,9,9,9,9, 12,12,12,12,12,12}","{12,12,12,12,12,12}","{12,12,12,12,12,12}","{5,9,9,9,9,9, 12,12,12,12,12,12}","{12,12,12,12,12,12}","{5,9,9,9,9,9, 12,12,12,12,12,12}","{5,9,9,9,9,9, 12,12,12,12,12,12, 54,54,54,54,5..."
7,{},{},{},{},{},{},{},"{54,54,54,54,54,54,54}",{},{},{},{},{},{},{},"{54,54,54,54,54,54,54}"
8,{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}
9,{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}
10,{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}


In [59]:
# ngram set -> ngram 개수
overlap_ngram_num = pd.DataFrame(columns=["N","nginx","httpd","tomcat","node","mongodb","mysql","mariadb","redis","gcc","openjdk","gzip","bzip2","qalc","ghostscript","lowriter","total"])

for index, row in overlap_ngram.iterrows():
    overlaptmp = {"N":index, "nginx":len(row.nginx),"httpd":len(row.httpd),"tomcat":len(row.tomcat),"node":len(row.node),
                  "mongodb":len(row.mongodb),"mysql":len(row.mysql),"mariadb":len(row.mariadb),"redis":len(row.redis),
                  "gcc":len(row.gcc),"openjdk":len(row.openjdk),"gzip":len(row.gzip),"bzip2":len(row.bzip2),"qalc":len(row.qalc),
                  "ghostscript":len(row.ghostscript),"lowriter":len(row.lowriter),"total":len(row.total)}
    overlap_ngram_num = overlap_ngram_num.append(overlaptmp, ignore_index=True)

# excel로 저장
overlap_ngram_num.to_excel("overlap_ngram_num.xlsx")
overlap_ngram_num

,N,nginx,httpd,tomcat,node,mongodb,mysql,mariadb,redis,gcc,openjdk,gzip,bzip2,qalc,ghostscript,lowriter,total
0,2,3,12,7,4,19,22,11,9,34,50,9,9,48,36,71,82
1,3,1,2,3,2,6,11,2,3,16,18,13,8,14,21,28,45
2,4,0,0,0,0,1,5,0,0,5,5,4,3,5,6,7,9
3,5,0,0,0,0,0,2,0,0,3,3,0,1,3,2,4,4
4,6,0,0,0,0,0,0,0,1,1,2,1,1,2,1,2,3
5,7,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
6,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,11,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1


# Exploit - N-gram Range 조사

In [19]:
import pandas as pd
m_ratio = pd.DataFrame(columns=["N","Ngram","Ngram_num","Exploit","Exploit_num","Ratio"])

# exploit path 정리 (list에서 str으로 변환)
m_path=dict()
for eid, pathset in eid_all_path_set.items():
    tmppath = list()
    for path in  pathset:
        tmppath.append(",".join(path))
    m_path[eid]=tmppath
m_ngram

,N,Ngram_syscall,num_of_exploit,EIDs
0,61,"[41, 248, 248, 248, 54, 0, 3, 5, 9, 10, 12, 13...",2,"[40871, 47170]"
1,49,"[230, 0, 3, 5, 9, 10, 12, 13, 14, 17, 56, 218,...",2,"[40871, 47170]"
2,45,"[12, 12, 5, 0, 257, 4, 12, 12, 5, 0, 257, 4, 1...",2,"[47163, 50541]"
3,43,"[9, 0, 3, 5, 9, 10, 12, 13, 14, 17, 56, 218, 2...",2,"[40616, 43127]"
4,43,"[12, 12, 5, 0, 257, 4, 12, 12, 5, 0, 257, 4, 1...",2,"[47163, 50541]"
...,...,...,...,...
473,2,"[257, 0]",16,"[15024, 15774, 3, 35370, 40616, 40871, 42275, ..."
474,2,"[8, 0]",8,"[12, 20979, 27297, 3, 43127, 45553, 50808, 9083]"
475,2,"[3, 59]",9,"[39230, 40871, 43418, 47170, 6851, 9479, 9542]"
476,2,"[8, 5]",4,"[12, 3, 43127, 50541]"


In [20]:
# 오름차순으로 정렬
m_ngram = m_ngram.sort_values(by=["N"], ascending=True)
block_ngram = dict()
block_EIDs = dict()
for N in set(m_ngram.N.to_list()):
    block_ngram[N]=set()
    block_EIDs[N]=set()
for index, row in m_ngram.iterrows():
    block_ngram[row.N].add(",".join(row.Ngram_syscall))
    block_EIDs[row.N] = block_EIDs[row.N] | set(row.EIDs)

In [26]:
# 2-gram exploit 1개 추가
block_EIDs[2] = block_EIDs[2] | {"0"}

### Range
- x <= N (of N-gram Pattern) <= y
- 7 <= x <= 65
- x <= y <= 65

In [27]:
# M(exploit) coverage 조사
##############################
# range 설정
X = 2
Y = 61
##############################

m_ratio = pd.DataFrame(columns=["N","Ngram","Ngram_num","Range_ngram","Range_ngram_num","Exploit","Exploit_num","Range_exploit","Range_exploit_num","Range_ratio"])
TOTAL=106
includedEidSet = set()
includedNgramSet = set()

for N in set(m_ngram.N.to_list()):
    if N < X: # N이 X가 될 때까지 continue
        continue
    
    tmpratio = {"N":0,"Ngram":[],"Ngram_num":0,"Range_ngram":set(),"Range_ngram_num":0,"Exploit":set(),"Exploit_num":0,"Range_exploit":set(),"Range_exploit_num":0,"Range_ratio":0}    
    tmpratio["N"] = N
    tmpratio["Ngram"] = block_ngram[N]
    tmpratio["Ngram_num"] = len(block_ngram[N])
    includedNgramSet = includedNgramSet | block_ngram[N]
    tmpratio["Range_ngram"] = includedNgramSet
    tmpratio["Range_ngram_num"] = len(includedNgramSet)
    tmpratio["Exploit"] = block_EIDs[N]
    tmpratio["Exploit_num"] = len(block_EIDs[N])
    includedEidSet = includedEidSet | block_EIDs[N]
    tmpratio["Range_exploit"] = includedEidSet
    tmpratio["Range_exploit_num"] = len(includedEidSet)
    tmpratio["Range_ratio"] = len(includedEidSet)/TOTAL*100

    # pd에 append
    m_ratio = m_ratio.append(tmpratio, ignore_index=True)
# 엑셀에 저장
m_ratio.to_excel(f"exploit_ratio_from{X}to{Y}_ascending.xlsx")
m_ratio

,N,Ngram,Ngram_num,Range_ngram,Range_ngram_num,Exploit,Exploit_num,Range_exploit,Range_exploit_num,Range_ratio
0,2,"{41,41, 257,5, 42,54, 12,56, 9,11, 3,257, 53,5...",163,"{41,41, 257,5, 42,54, 12,56, 9,11, 3,257, 53,5...",163,"{9545, 25202, 4756, 20721, 43029, 43127, 33322...",106,"{9545, 25202, 4756, 20721, 43029, 43127, 33322...",106,100.0
1,3,"{1,0,1, 103,9,103, 16,16,16, 3,3,1, 9,257,59, ...",108,"{41,41, 257,5, 1,0,1, 42,54, 12,56, 9,11, 103,...",271,"{9545, 25202, 4756, 20721, 43127, 43029, 33322...",92,"{4756, 20721, 43029, 33322, 9083, 33589, 37167...",106,100.0
2,4,"{293,72,1,56, 9,3,257,0, 257,3,56,59, 102,107,...",52,"{293,72,1,56, 42,54, 103,9,103, 3,257, 53,54, ...",323,"{25202, 20721, 43127, 43029, 33322, 9083, 3351...",78,"{4756, 20721, 43029, 33322, 9083, 33589, 37167...",106,100.0
3,5,"{5,5,8,0,8, 12,12,257,5,0, 9,9,9,9,9, 56,257,1...",18,"{293,72,1,56, 5,5,8,0,8, 42,54, 103,9,103, 3,2...",341,"{15704, 17787, 39230, 25202, 40839, 20721, 430...",49,"{4756, 20721, 43029, 33322, 9083, 33589, 37167...",106,100.0
4,6,"{12,12,257,0,5,3, 257,5,12,12,0,3, 12,12,257,5...",20,"{293,72,1,56, 5,5,8,0,8, 42,54, 103,9,103, 3,2...",361,"{15704, 17787, 39166, 42887, 40839, 2013, 2072...",54,"{4756, 20721, 43029, 33322, 9083, 33589, 37167...",106,100.0
5,7,"{257,5,12,12,217,217,3, 5,0,5,0,5,0,3, 3,56,61...",9,"{293,72,1,56, 5,5,8,0,8, 42,54, 103,9,103, 3,2...",370,"{15704, 17787, 20721, 43029, 33516, 50541, 3, ...",34,"{9545, 25202, 4756, 20721, 43029, 43127, 33322...",106,100.0
6,8,"{41,12,12,9,54,54,54,3, 101,61,101,61,101,101,...",8,"{293,72,1,56, 5,5,8,0,8, 42,54, 103,9,103, 3,2...",378,"{15704, 17787, 20721, 43029, 43127, 33516, 505...",34,"{9545, 25202, 4756, 20721, 43029, 43127, 33322...",106,100.0
7,9,"{12,12,257,12,12,257,5,0,5, 12,12,257,5,0,5,0,...",6,"{293,72,1,56, 5,5,8,0,8, 42,54, 103,9,103, 3,2...",384,"{15704, 17787, 39230, 40839, 43029, 9083, 3351...",27,"{9545, 25202, 4756, 20721, 43029, 43127, 33322...",106,100.0
8,10,"{3,12,12,257,5,0,5,0,5,0, 5,9,9,9,9,3,10,11,20...",6,"{293,72,1,56, 5,5,8,0,8, 42,54, 12,12,257,5,0,...",390,"{15704, 17787, 20721, 47170, 43029, 15916, 471...",27,"{9545, 25202, 4756, 20721, 43029, 43127, 33322...",106,100.0
9,11,"{257,4,230,257,4,230,257,4,230,257,4, 41,54,54...",5,"{293,72,1,56, 5,5,8,0,8, 42,54, 12,12,257,5,0,...",395,"{15704, 17787, 39230, 47170, 43029, 15916, 895...",25,"{9545, 25202, 4756, 20721, 43029, 43127, 33322...",106,100.0


### 내림차순 조사

In [60]:
# 오름차순으로 정렬
m_ngram = m_ngram.sort_values(by=["N"], ascending=False)
block_ngram = dict()
block_EIDs = dict()
for N in set(m_ngram.N.to_list()):
    block_ngram[N]=set()
    block_EIDs[N]=set()
for index, row in m_ngram.iterrows():
    block_ngram[row.N].add(",".join(row.Ngram_syscall))
    block_EIDs[row.N] = block_EIDs[row.N] | set(row.EIDs)

In [61]:
# 2-gram exploit 1개 추가
block_EIDs[2] = block_EIDs[2] | {"0"}

In [62]:
# M(exploit) coverage 조사 - 내림차순

m_ratio = pd.DataFrame(columns=["N","Ngram","Ngram_num","Range_ngram","Range_ngram_num","Exploit","Exploit_num","Range_exploit","Range_exploit_num","Range_ratio"])
TOTAL=106
includedEidSet = set()
includedNgramSet = set()

N_list = list(set(m_ngram.N.to_list()))
N_list.sort(reverse=True)

for N in N_list: 
    tmpratio = {"N":0,"Ngram":[],"Ngram_num":0,"Range_ngram":set(),"Range_ngram_num":0,"Exploit":set(),"Exploit_num":0,"Range_exploit":set(),"Range_exploit_num":0,"Range_ratio":0}    
    tmpratio["N"] = N
    tmpratio["Ngram"] = block_ngram[N]
    tmpratio["Ngram_num"] = len(block_ngram[N])
    includedNgramSet = includedNgramSet | block_ngram[N]
    tmpratio["Range_ngram"] = includedNgramSet
    tmpratio["Range_ngram_num"] = len(includedNgramSet)
    tmpratio["Exploit"] = block_EIDs[N]
    tmpratio["Exploit_num"] = len(block_EIDs[N])
    includedEidSet = includedEidSet | block_EIDs[N]
    tmpratio["Range_exploit"] = includedEidSet
    tmpratio["Range_exploit_num"] = len(includedEidSet)
    tmpratio["Range_ratio"] = len(includedEidSet)/TOTAL*100

    # pd에 append
    m_ratio = m_ratio.append(tmpratio, ignore_index=True)
# # 엑셀에 저장
m_ratio.to_excel(f"exploit_ratio_dexcending.xlsx")
m_ratio

,N,Ngram,Ngram_num,Range_ngram,Range_ngram_num,Exploit,Exploit_num,Range_exploit,Range_exploit_num,Range_ratio
0,61,"{41,248,248,248,54,0,3,5,9,10,12,13,14,17,56,2...",1,"{41,248,248,248,54,0,3,5,9,10,12,13,14,17,56,2...",1,"{47170, 40871}",2,"{47170, 40871}",2,1.886792
1,49,"{230,0,3,5,9,10,12,13,14,17,56,218,257,273,302...",1,"{230,0,3,5,9,10,12,13,14,17,56,218,257,273,302...",2,"{47170, 40871}",2,"{47170, 40871}",2,1.886792
2,45,"{12,12,5,0,257,4,12,12,5,0,257,4,12,12,5,0,257...",1,"{230,0,3,5,9,10,12,13,14,17,56,218,257,273,302...",3,"{47163, 50541}",2,"{47170, 47163, 40871, 50541}",4,3.773585
3,43,"{9,0,3,5,9,10,12,13,14,17,56,218,257,273,302,0...",2,"{230,0,3,5,9,10,12,13,14,17,56,218,257,273,302...",5,"{40616, 43127, 47163, 50541}",4,"{47170, 43127, 47163, 40871, 40616, 50541}",6,5.660377
4,42,"{0,3,5,9,10,12,13,14,17,56,218,257,273,302,0,3...",1,"{230,0,3,5,9,10,12,13,14,17,56,218,257,273,302...",6,"{40616, 43127, 44302}",3,"{44302, 47170, 43127, 47163, 40871, 40616, 50541}",7,6.603774
5,39,"{12,12,5,0,257,4,5,0,257,4,5,0,257,4,5,0,61,13...",1,"{230,0,3,5,9,10,12,13,14,17,56,218,257,273,302...",7,"{47163, 50541}",2,"{44302, 47170, 43127, 47163, 40871, 40616, 50541}",7,6.603774
6,38,"{12,12,257,12,12,257,5,0,5,0,5,0,5,0,5,0,5,0,3...",1,"{12,12,257,12,12,257,5,0,5,0,5,0,5,0,5,0,5,0,3...",8,"{14814, 15023}",2,"{44302, 47170, 43127, 15023, 47163, 40871, 406...",9,8.490566
7,35,"{12,12,257,12,12,257,5,0,5,0,5,0,5,0,5,0,5,0,3...",1,"{230,0,3,5,9,10,12,13,14,17,56,218,257,273,302...",9,"{14814, 15023}",2,"{44302, 47170, 43127, 15023, 47163, 40871, 406...",9,8.490566
8,34,"{12,12,257,5,0,5,0,5,0,5,0,5,0,5,0,3,12,12,257...",1,"{230,0,3,5,9,10,12,13,14,17,56,218,257,273,302...",10,{15023},1,"{44302, 47170, 43127, 15023, 47163, 40871, 406...",9,8.490566
9,33,"{12,12,257,5,0,5,0,5,0,5,0,5,0,5,0,3,12,12,257...",2,"{230,0,3,5,9,10,12,13,14,17,56,218,257,273,302...",12,"{42275, 47163, 44303, 15023, 50541}",5,"{42275, 44302, 47170, 43127, 15023, 47163, 408...",11,10.377358


In [48]:
seq = "257,5,9,3,257,0,5,9,9,9,9"

for img, imgseq in iter_seqNumList.items():
        if seq in ",".join(imgseq):
            print(img)

mysql_c


In [73]:
# tmplist = ['12', '12', '87', '1', '1', '1', '5', '9', '9', '9', '9', '9', '160', '59'] # 45516 - length:14
tmplist = ['149', '149', '144', '203', '257', '257', '8', '0', '8', '1'] #27297 - length:10
# tmplist = ['39', '9', '102', '104', '117', '119', '56', '257', '3', '257', '3', '257', '3', '59'] #33322 - length:14
print(len(tmplist))
for sys in tmplist:
    print(sys+":"+sysnum_convert[sys])

10
149:mlock
149:mlock
144:sched_setscheduler
203:sched_setaffinity
257:openat
257:openat
8:lseek
0:read
8:lseek
1:write
